# And

To do classical logic with a reversible circuit (a pre-requisite for a quantum circuit), we use a three (qu)bit operation called a Toffli gate that takes `[a, b, c]` to `[a, b, c ^ (a & b)]`. If we take `c` to be zero, this is an And gate taking `[a, b]` to `[a, b, a & b]`.

In [ ]:
import itertools
for a, b, in itertools.product([0, 1], repeat=2):
    print(a, b, '->', a & b)

## Quantum operation

We provide a quantum operation for performing quantum And. Specifically, it assumes the third qubit (i.e. the target) is initialized to the `|0>` state.

In [ ]:
import cirq
from cirq.contrib.svg import SVGCircuit
from cirq_qubitization.and_gate import And

qa, qb, qout = cirq.LineQubit.range(3)
circuit = cirq.Circuit(And().on(qa, qb, qout))
SVGCircuit(circuit)

## Efficient decomposition

This specialization of the Toffli gate permits a specialized decomposition that minimizes the `T`-gate count.

In [ ]:
c2 = cirq.Circuit(cirq.decompose_once(And().on(qa, qb, qout)))
SVGCircuit(c2)

## Test behavior

We can test the behavior of the And gate on computational basis states using a Quantum simulator.

In [ ]:
input_states = [(a, b, 0) for a, b in itertools.product([0, 1], repeat=2)]
output_states = [(a, b, a & b) for a, b, _ in input_states]


for input, output in zip(input_states, output_states):
    result = cirq.Simulator().simulate(circuit, initial_state=input)
    print(input, '->', result.dirac_notation())
    assert result.dirac_notation()[1:-1] == "".join(str(x) for x in output)

In [ ]:
import numpy as np
inds, = np.where(result.final_state_vector)
assert len(inds) == 1
ind, = inds
f'{ind:3b}'

## Uncompute

We can save even more `T` gates when "uncomputing" an And operation, i.e. performing the adjoint operation by using classical control.

In [ ]:
op = And().on(qa, qb, qout)
op = op ** -1
circuit = cirq.Circuit(op)
SVGCircuit(circuit)

Note that we reset our target using measurement and fix up phases depending on the result of that measurement.

In [ ]:
c2 = cirq.Circuit(cirq.decompose_once(And(adjoint=True).on(qa, qb, qout)))
c2

## Test Adjoint

In [ ]:
input_states = [(a, b, a & b) for a, b in itertools.product([0, 1], repeat=2)]
output_states = [(a, b, 0) for a, b, _ in input_states]

circuit = cirq.Circuit(And(adjoint=True).on(qa, qb, qout))
for input, output in zip(input_states, output_states):
    result = cirq.Simulator().simulate(circuit, initial_state=input)
    print(input, '->', result.dirac_notation())
    assert result.dirac_notation()[1:-1] == "".join(str(x) for x in output)